# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

df = pd.read_csv("..\output_data\my_weather_data.csv")

df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,71.56,68,20,14.12,TO,1593788400
1,1,Atuona,-9.8000,-139.0333,80.56,77,4,25.30,PF,1593788400
2,2,Poddor'ye,57.4695,31.1156,74.77,46,59,4.79,RU,1593788400
3,3,Hong Kong,22.2855,114.1577,84.11,82,69,11.83,HK,1593788400
4,4,Mataura,-46.1927,168.8643,39.99,78,100,6.44,NZ,1593788400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Latitude", "Longitude"]].astype(float)

humidity_rate = df["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heatmap_layer)

plt.savefig('..\output_data\HeatMap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
conditional_df = df.loc[df["Max Temp"]<80, :]
conditional_df = conditional_df.loc[conditional_df["Max Temp"]>70,:]
conditional_df.count()

Unnamed: 0    147
City          147
Latitude      147
Longitude     147
Max Temp      147
Humidity      147
Cloudiness    147
Wind Speed    147
Country       144
Date          147
dtype: int64

In [7]:
conditional_df = conditional_df.loc[conditional_df["Wind Speed"]<10,:]
conditional_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Poddor'ye,57.4695,31.1156,74.77,46,59,4.79,RU,1593788400
10,10,Albany,42.6001,-73.9662,73.42,82,96,3.38,US,1593788400
15,15,Kapaa,22.0752,-159.3190,76.60,78,50,8.05,US,1593788400
18,18,Carnarvon,-24.8667,113.6333,73.72,63,0,7.20,AU,1593788400
19,19,Ribeira Grande,38.5167,-28.7000,70.66,80,34,6.13,PT,1593788400


In [8]:
conditional_df = conditional_df.loc[conditional_df["Cloudiness"]==0,:]
conditional_df.head(20)

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Carnarvon,-24.8667,113.6333,73.72,63,0,7.20,AU,1593788400
155,155,Guerrero Negro,27.9769,-114.0611,72.91,62,0,5.59,MX,1593788400
173,173,Dekar,-21.5333,21.9333,73.87,20,0,8.14,BW,1593788400
244,244,Labuhan,-6.8844,112.2051,79.05,80,0,8.66,ID,1593788400
330,330,Menongue,-14.6585,17.6910,78.12,18,0,8.03,AO,1593788400
452,452,Thohoyandou,-22.9456,30.4850,73.99,27,0,6.04,ZA,1593788400
491,491,Obigarm,38.7173,69.7088,76.62,25,0,4.63,TJ,1593788400
499,499,Moosomin,50.1333,-101.6677,71.35,71,0,4.09,CA,1593788400


In [9]:
# No null values so no need to drop anything :-) 

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = pd.DataFrame()
hotel_df['City'] = conditional_df['City']
hotel_df['Country'] = conditional_df['Country']
hotel_df['Lat'] = conditional_df['Latitude']
hotel_df['Lng'] = conditional_df['Longitude']

hotel_df.head()

,City,Country,Lat,Lng
18,Carnarvon,AU,-24.8667,113.6333
155,Guerrero Negro,MX,27.9769,-114.0611
173,Dekar,BW,-21.5333,21.9333
244,Labuhan,ID,-6.8844,112.2051
330,Menongue,AO,-14.6585,17.6910


In [11]:
for index, row in hotel_df.iterrows():

    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']
    
# url and API request
   
    req = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}")
    
    response = req.json()
    
    results = response['results']
    print(f"Retrieving Results for  {city}.")
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    

Retrieving Results for  Carnarvon.
Closest hotel to Carnarvon at -24.8667 , 113.6333 is Carnarvon Tyrepower.
Retrieving Results for  Guerrero Negro.
Closest hotel to Guerrero Negro at 27.9769 , -114.0611 is Empresas Matco.
Retrieving Results for  Dekar.
Missing field/result... skipping.
Retrieving Results for  Labuhan.
Closest hotel to Labuhan at -6.8844 , 112.2051 is Pondok Modern Darul Islam.
Retrieving Results for  Menongue.
Closest hotel to Menongue at -14.6585 , 17.691 is BANCO ECONÓMICO MENONGUE.
Retrieving Results for  Thohoyandou.
Closest hotel to Thohoyandou at -22.9456 , 30.485 is Blue IT Marketing PTY LTD.
Retrieving Results for  Obigarm.
Closest hotel to Obigarm at 38.7173 , 69.7088 is Park Imeni Ismoila Somoni.
Retrieving Results for  Moosomin.
Closest hotel to Moosomin at 50.1333 , -101.6677 is Royal Canadian Legion Branch 81.


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:

# Add marker
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

plt.savefig('..\output_data\VacationMap.png')
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>